# 비비빅 학습동아리 12회차_스터디
 - 2024.9.24.(수) 18:10~20:30
 - 국감대비 엑셀 to 파이썬

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

- 엑셀파일을 먼저 열어서 살펴봅니다.
    - 총 4개 시트, 데이터 갯수 등등.
    - 만약에 이 데이터들이 100만개가 넘는다면?;;;; vlookup 할수있을까?
    - 단순 pivot이 아닌, 중복제거, 데이터 집계 등 작업이 필요한 경우? 어떻게 할건가?

In [2]:
# 엑셀 연계를 위해서는 openpyxl 라이브러리가 필요함
# 아나콘다 설치 시 함께 깔림
#!pip install openpyxl 

In [3]:
# 엑셀 파일 열기 (리소스 할당)
path = './data/sample_data.xlsx'
xls = pd.ExcelFile(path)

# 파일 닫기 (리소스 할당 해제)
# xls_file.close()

In [4]:
# sheet_names를 통해 시트명을 불러올 수 있음
sheets = xls.sheet_names
sheets  # 시트 리스트로 저장

['People', 'Hospitals', 'Diseases', 'Visits']

In [5]:
# parse를 통해 df로 불러올 수 있음
# 기본형 - 시트명을 지정
df = xls.parse('People')
df.head(5)
# 시트 리스트에서 불러오기
# df = xls.parse(sheets[1])

,이름,주민번호,생년월일,성별,주소
0,손지은,890924-2790918,1989-09-24,F,대구광역시 도봉구 선릉길
1,박정식,990924-1256214,1999-09-24,M,전라남도 군포시 석촌호수가 (재현최최동)
2,김서영,000923-4366119,2000-09-23,F,서울특별시 서초구 봉은사가
3,최성진,650924-2668495,1965-09-24,F,전라북도 용인시 도산대로
4,서상훈,650924-1635497,1965-09-24,M,충청남도 용인시 서초대406로 (서윤이오마을)


In [6]:
xls.close()

In [7]:
xls

In [8]:
# 엑셀 파일 열기 (리소스 할당)
path = './data/sample_data_hard.xlsx'
xls = pd.ExcelFile(path)

sheets = xls.sheet_names
sheets  # 총 4개의 시트가 있음을 확인  ['People', 'Hospitals', 'Diseases', 'Visits']

['People', 'Hospitals', 'Diseases', 'Visits']

In [9]:
# csv처럼 저장되지 않은 엑셀 데이터 불러오기
df = xls.parse('People')
# 엑셀에 제목이 들어있는 등 데이터의 문제가 많음
df.head(5)

,수진자 데이터,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,이름,주민번호,생년월일,성별,주소
2,손지은,890924-2790918,1989-09-24,F,대구광역시 도봉구 선릉길
3,박정식,990924-1256214,1999-09-24,M,전라남도 군포시 석촌호수가 (재현최최동)
4,김서영,000923-4366119,2000-09-23,F,서울특별시 서초구 봉은사가


In [10]:
#skiprows 사용. 맨 위 k개 행을 제외
df1 = xls.parse('People', skiprows=2)
df1

,이름,주민번호,생년월일,성별,주소
0,손지은,890924-2790918,1989-09-24,F,대구광역시 도봉구 선릉길
1,박정식,990924-1256214,1999-09-24,M,전라남도 군포시 석촌호수가 (재현최최동)
2,김서영,000923-4366119,2000-09-23,F,서울특별시 서초구 봉은사가
3,최성진,650924-2668495,1965-09-24,F,전라북도 용인시 도산대로
4,서상훈,650924-1635497,1965-09-24,M,충청남도 용인시 서초대406로 (서윤이오마을)
...,...,...,...,...,...
295,전서준,850924-2063946,1985-09-24,F,경기도 양구군 역삼로
296,최성민,010924-4412885,2001-09-24,F,울산광역시 서구 개포가
297,박도윤,730924-1060622,1973-09-24,M,충청북도 제천시 논현로 (채원최읍)
298,강혜진,510924-1038271,1951-09-24,M,세종특별자치시 노원구 언주90로 (현준한읍)


In [11]:
#usecols 사용하여, 제목셀을 무시
df2 = xls.parse(sheets[1], skiprows=3, usecols='A:J')
df2

,요양기관기호,병원이름,의사,면허번호,주소,의사 수,간호사 수,행정 직원 수,물리치료사 수,방사선사 수
0,29779167,(유) 김병원,류순자,91908,세종특별자치시 서구 봉은사550길,9,17,3,3,3
1,24259773,(주) 이최김병원,김영희,58900,경상남도 단양군 양재천길,8,4,3,15,3
2,23634673,김김이병원,이영길,37694,충청남도 광주시 영동대로 (영환이면),4,12,0,5,0
3,30890242,주식회사 김병원,류상훈,83427,대구광역시 동구 양재천길,7,7,4,16,2
4,17850480,주식회사 이이김병원,오지은,10372,전라남도 안산시 상록구 영동대가 (아름최읍),8,29,4,22,3
...,...,...,...,...,...,...,...,...,...,...
95,37617862,(주) 배송김병원,오주원,65536,경상남도 과천시 언주길 (성훈오홍읍),7,15,1,28,3
96,32360684,(주) 이병원,오경수,13142,충청남도 청양군 영동대거리,2,7,4,12,5
97,12189280,권손배병원,박준영,86768,인천광역시 서대문구 가락거리 (영호김읍),10,11,1,10,2
98,38116308,유한회사 전병원,김영길,85000,경기도 홍천군 봉은사538로,8,3,0,15,2


In [12]:
# 상병코드 불러오기
# 더 복잡한 형태
df3 = xls.parse(sheets[2], skiprows=5)
df3
# 연번은 index로 처리
# Unnamed: 3	Unnamed: 4	Unnamed: 5	Unnamed: 6 불러오지 않기
# 하단 오류 데이터는 제거

,연번,병명코드,병명,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1,MW510006,고혈압,NaN,NaN,NaN,NaN
1,2,FY072196,당뇨병,NaN,NaN,NaN,NaN
2,3,IV815605,천식,NaN,NaN,NaN,NaN
3,4,VE824665,기관지염,NaN,NaN,NaN,NaN
4,5,WZ383794,폐렴,NaN,NaN,NaN,NaN
5,6,TF244363,결핵,NaN,NaN,NaN,NaN
6,7,ZM653921,알츠하이머병,NaN,NaN,NaN,NaN
7,8,SI033180,파킨슨병,NaN,NaN,NaN,NaN
8,9,FO539167,간경변,NaN,NaN,NaN,NaN
9,10,NA310640,심부전,NaN,NaN,NaN,NaN


In [13]:
#최초 nan 값 찾기
df3.연번.isna().idxmax()  ## 30


30

In [14]:
# index_col, usecols, skipfooter 사용  ## 43-29 = 13
df3 = xls.parse(sheets[2], 
                skiprows=5, 
                #index_col='연번', 
                index_col=[0],
                usecols='B,C',   # usecols='A,C' , 'A:D', 'A, C:D, G', 'A,B,C'
                skipfooter=14)
df3


,병명
병명코드,
MW510006,고혈압
FY072196,당뇨병
IV815605,천식
VE824665,기관지염
WZ383794,폐렴
TF244363,결핵
ZM653921,알츠하이머병
SI033180,파킨슨병
FO539167,간경변


In [15]:
# 진료내역은
# skiprows 사용
df4 = xls.parse(sheets[3], skiprows=2)
df4


,진료일자,주민번호,요양기관기호,병명코드,진료비
0,2022-05-19,600923-2436130,24783278,SR868470,340000
1,2022-04-13,730924-2848893,27643140,TF244363,460000
2,2022-07-03,070924-3517526,19385500,IV815605,440000
3,2021-02-07,730924-1397414,29623568,MW510006,310000
4,2020-11-19,750924-2247494,26061768,ID709345,310000
...,...,...,...,...,...
2995,2022-04-14,140924-3619479,17135290,ZM653921,50000
2996,2024-11-18,910924-1963129,28915603,HZ821903,480000
2997,2023-06-22,950924-2827978,31939156,SG977307,340000
2998,2020-07-29,050924-4577909,14914413,FP208641,40000


# CASE study

## 1. vlookup
진료내역에 수진자 이름, 생년월일, 성별, 주소를 붙여보자

In [16]:
# 빈 데이터프레임
df_exm = pd.DataFrame()
df_exm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame


In [17]:
# df4.merge(df1)
# 주민번호가 같기 때문에 자동으로 붙는다.
# 명시적으로 붙여주는 방법 (이걸 추천)
df_exm = df4.merge(df1, how='left', left_on='주민번호', right_on='주민번호')
df_exm

,진료일자,주민번호,요양기관기호,병명코드,진료비,이름,생년월일,성별,주소
0,2022-05-19,600923-2436130,24783278,SR868470,340000,김정자,1960-09-23,F,대전광역시 광진구 도산대거리 (은경이강읍)
1,2022-04-13,730924-2848893,27643140,TF244363,460000,이예지,1973-09-24,F,부산광역시 서대문구 테헤란길 (경수홍면)
2,2022-07-03,070924-3517526,19385500,IV815605,440000,김영식,2007-09-24,M,경기도 가평군 서초중앙가 (서연허양면)
3,2021-02-07,730924-1397414,29623568,MW510006,310000,이채원,1973-09-24,M,세종특별자치시 은평구 개포길 (예은김양마을)
4,2020-11-19,750924-2247494,26061768,ID709345,310000,정미경,1975-09-24,F,경상남도 청주시 서원구 개포거리 (경숙김황마을)
...,...,...,...,...,...,...,...,...,...
2995,2022-04-14,140924-3619479,17135290,ZM653921,50000,김옥순,2014-09-24,M,대구광역시 서대문구 논현00거리 (지은김김마을)
2996,2024-11-18,910924-1963129,28915603,HZ821903,480000,김서윤,1991-09-24,M,대구광역시 강서구 학동길
2997,2023-06-22,950924-2827978,31939156,SG977307,340000,박윤서,1995-09-24,F,제주특별자치도 용인시 수지구 테헤란로
2998,2020-07-29,050924-4577909,14914413,FP208641,40000,배윤서,2005-09-24,F,대전광역시 서초구 영동대가


In [18]:
df_exm = df_exm.merge(df2, how='left', left_on='요양기관기호', right_on = '요양기관기호'  )
df_exm
# 중복실행 문제를 방지하려면, suffix 옵션을 사용한다.
# 오늘 다루지는 않겠음 -> gpt문의하면 잘 알려줘요
# 이런 에러가 생긴 경우, df_exm을 처음부터 재구축해주자. df_exm = pd.DataFrame()

,진료일자,주민번호,요양기관기호,병명코드,진료비,이름,생년월일,성별,주소_x,병원이름,의사,면허번호,주소_y,의사 수,간호사 수,행정 직원 수,물리치료사 수,방사선사 수
0,2022-05-19,600923-2436130,24783278,SR868470,340000,김정자,1960-09-23,F,대전광역시 광진구 도산대거리 (은경이강읍),(유) 고병원,김미경,76401,경상남도 음성군 잠실가 (정수이하읍),5,2,5,8,2
1,2022-04-13,730924-2848893,27643140,TF244363,460000,이예지,1973-09-24,F,부산광역시 서대문구 테헤란길 (경수홍면),주식회사 박병원,김진호,56132,경상남도 평창군 오금길,10,18,1,28,0
2,2022-07-03,070924-3517526,19385500,IV815605,440000,김영식,2007-09-24,M,경기도 가평군 서초중앙가 (서연허양면),유한회사 장병원,박서윤,42205,제주특별자치도 시흥시 가락3가,1,3,0,2,0
3,2021-02-07,730924-1397414,29623568,MW510006,310000,이채원,1973-09-24,M,세종특별자치시 은평구 개포길 (예은김양마을),주식회사 김이박병원,강영식,22453,서울특별시 구로구 삼성14가,2,3,4,2,2
4,2020-11-19,750924-2247494,26061768,ID709345,310000,정미경,1975-09-24,F,경상남도 청주시 서원구 개포거리 (경숙김황마을),주식회사 김양백병원,신예원,90772,전라남도 안산시 단원구 반포대3길,9,1,4,26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2022-04-14,140924-3619479,17135290,ZM653921,50000,김옥순,2014-09-24,M,대구광역시 서대문구 논현00거리 (지은김김마을),신박병원,김미경,90098,제주특별자치도 부천시 소사구 가락가,2,17,2,25,3
2996,2024-11-18,910924-1963129,28915603,HZ821903,480000,김서윤,1991-09-24,M,대구광역시 강서구 학동길,이김윤병원,송성현,39608,경기도 태백시 서초대로 (병철곽정읍),2,2,2,24,0
2997,2023-06-22,950924-2827978,31939156,SG977307,340000,박윤서,1995-09-24,F,제주특별자치도 용인시 수지구 테헤란로,조김강병원,황서현,28566,제주특별자치도 여주시 언주46로,6,15,4,24,4
2998,2020-07-29,050924-4577909,14914413,FP208641,40000,배윤서,2005-09-24,F,대전광역시 서초구 영동대가,유한회사 주병원,김은경,20199,인천광역시 중랑구 테헤란길,6,9,3,21,2


In [19]:
# df3에서 병명코드를 index로 부여했으므로, right_on = '병명코드' 대신 right_index = True 옵션을 사용함.
df_exm = df_exm.merge(df3, how='left', left_on='병명코드', right_index = True  )
df_exm
# 중복실행 문제를 방지하려면, suffix 옵션을 사용한다.
# 오늘 다루지는 않겠음 -> gpt문의하면 잘 알려줘요
# 이런 에러가 생긴 경우, df_exm을 처음부터 재구축해주자. df_exm = pd.DataFrame()

,진료일자,주민번호,요양기관기호,병명코드,진료비,이름,생년월일,성별,주소_x,병원이름,의사,면허번호,주소_y,의사 수,간호사 수,행정 직원 수,물리치료사 수,방사선사 수,병명
0,2022-05-19,600923-2436130,24783278,SR868470,340000,김정자,1960-09-23,F,대전광역시 광진구 도산대거리 (은경이강읍),(유) 고병원,김미경,76401,경상남도 음성군 잠실가 (정수이하읍),5,2,5,8,2,간암
1,2022-04-13,730924-2848893,27643140,TF244363,460000,이예지,1973-09-24,F,부산광역시 서대문구 테헤란길 (경수홍면),주식회사 박병원,김진호,56132,경상남도 평창군 오금길,10,18,1,28,0,결핵
2,2022-07-03,070924-3517526,19385500,IV815605,440000,김영식,2007-09-24,M,경기도 가평군 서초중앙가 (서연허양면),유한회사 장병원,박서윤,42205,제주특별자치도 시흥시 가락3가,1,3,0,2,0,천식
3,2021-02-07,730924-1397414,29623568,MW510006,310000,이채원,1973-09-24,M,세종특별자치시 은평구 개포길 (예은김양마을),주식회사 김이박병원,강영식,22453,서울특별시 구로구 삼성14가,2,3,4,2,2,고혈압
4,2020-11-19,750924-2247494,26061768,ID709345,310000,정미경,1975-09-24,F,경상남도 청주시 서원구 개포거리 (경숙김황마을),주식회사 김양백병원,신예원,90772,전라남도 안산시 단원구 반포대3길,9,1,4,26,0,갑상선암
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2022-04-14,140924-3619479,17135290,ZM653921,50000,김옥순,2014-09-24,M,대구광역시 서대문구 논현00거리 (지은김김마을),신박병원,김미경,90098,제주특별자치도 부천시 소사구 가락가,2,17,2,25,3,알츠하이머병
2996,2024-11-18,910924-1963129,28915603,HZ821903,480000,김서윤,1991-09-24,M,대구광역시 강서구 학동길,이김윤병원,송성현,39608,경기도 태백시 서초대로 (병철곽정읍),2,2,2,24,0,맹장염
2997,2023-06-22,950924-2827978,31939156,SG977307,340000,박윤서,1995-09-24,F,제주특별자치도 용인시 수지구 테헤란로,조김강병원,황서현,28566,제주특별자치도 여주시 언주46로,6,15,4,24,4,신부전
2998,2020-07-29,050924-4577909,14914413,FP208641,40000,배윤서,2005-09-24,F,대전광역시 서초구 영동대가,유한회사 주병원,김은경,20199,인천광역시 중랑구 테헤란길,6,9,3,21,2,위암


## 1-1. 지역구분 컬럼을 하나 만들어줍시다.

In [20]:
# 시도/군구 를 한꺼번에 뽑으려면?
#df_exm[['수진자_시도','수진자_구']] = df_exm.주소_x.str.split(' ', n=2, expand=True)[0,1]
df_exm[['수진자_시도','수진자_구']] = df_exm.주소_x.str.split(' ', n=2, expand=True)[[0, 1]]
df_exm[['병원_시도','병원_구']] = df_exm.주소_y.str.split(' ', n=2, expand=True)[[0, 1]]
df_exm

,진료일자,주민번호,요양기관기호,병명코드,진료비,이름,생년월일,성별,주소_x,병원이름,...,의사 수,간호사 수,행정 직원 수,물리치료사 수,방사선사 수,병명,수진자_시도,수진자_구,병원_시도,병원_구
0,2022-05-19,600923-2436130,24783278,SR868470,340000,김정자,1960-09-23,F,대전광역시 광진구 도산대거리 (은경이강읍),(유) 고병원,...,5,2,5,8,2,간암,대전광역시,광진구,경상남도,음성군
1,2022-04-13,730924-2848893,27643140,TF244363,460000,이예지,1973-09-24,F,부산광역시 서대문구 테헤란길 (경수홍면),주식회사 박병원,...,10,18,1,28,0,결핵,부산광역시,서대문구,경상남도,평창군
2,2022-07-03,070924-3517526,19385500,IV815605,440000,김영식,2007-09-24,M,경기도 가평군 서초중앙가 (서연허양면),유한회사 장병원,...,1,3,0,2,0,천식,경기도,가평군,제주특별자치도,시흥시
3,2021-02-07,730924-1397414,29623568,MW510006,310000,이채원,1973-09-24,M,세종특별자치시 은평구 개포길 (예은김양마을),주식회사 김이박병원,...,2,3,4,2,2,고혈압,세종특별자치시,은평구,서울특별시,구로구
4,2020-11-19,750924-2247494,26061768,ID709345,310000,정미경,1975-09-24,F,경상남도 청주시 서원구 개포거리 (경숙김황마을),주식회사 김양백병원,...,9,1,4,26,0,갑상선암,경상남도,청주시,전라남도,안산시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2022-04-14,140924-3619479,17135290,ZM653921,50000,김옥순,2014-09-24,M,대구광역시 서대문구 논현00거리 (지은김김마을),신박병원,...,2,17,2,25,3,알츠하이머병,대구광역시,서대문구,제주특별자치도,부천시
2996,2024-11-18,910924-1963129,28915603,HZ821903,480000,김서윤,1991-09-24,M,대구광역시 강서구 학동길,이김윤병원,...,2,2,2,24,0,맹장염,대구광역시,강서구,경기도,태백시
2997,2023-06-22,950924-2827978,31939156,SG977307,340000,박윤서,1995-09-24,F,제주특별자치도 용인시 수지구 테헤란로,조김강병원,...,6,15,4,24,4,신부전,제주특별자치도,용인시,제주특별자치도,여주시
2998,2020-07-29,050924-4577909,14914413,FP208641,40000,배윤서,2005-09-24,F,대전광역시 서초구 영동대가,유한회사 주병원,...,6,9,3,21,2,위암,대전광역시,서초구,인천광역시,중랑구


## 2. 데이터에 중복이 있는 지 확인하는 방법  
중복이 있는 지 확인하는것은 매우 중요함 (데이터 검증 영역)

In [21]:
df1[df1.duplicated()]
# 중복이 없습니다.

,이름,주민번호,생년월일,성별,주소


In [22]:
# 여러번 쓴다면 함수로 만들어보자
def check_dup(df):
    return df[df.duplicated()]


In [23]:
check_dup(df4)

,진료일자,주민번호,요양기관기호,병명코드,진료비


In [24]:
# 특정 컬럼을 기준으로 체크하는 함수
# keep옵션은 False로 해야, 전체 중복된 내용을 체크할 수 있음
# 중복은 2개 행 중복, keep=False가 없는 경우, 둘 중 하나만 반환함.
dup = df1[df1.duplicated(subset=['이름'], keep=False)]
dup

,이름,주민번호,생년월일,성별,주소
1,박정식,990924-1256214,1999-09-24,M,전라남도 군포시 석촌호수가 (재현최최동)
2,김서영,000923-4366119,2000-09-23,F,서울특별시 서초구 봉은사가
3,최성진,650924-2668495,1965-09-24,F,전라북도 용인시 도산대로
6,송영진,740924-1711304,1974-09-24,M,대구광역시 강북구 테헤란길
17,김영식,990924-2976872,1999-09-24,F,울산광역시 금천구 삼성65로
...,...,...,...,...,...
276,이수빈,000923-4897416,2000-09-23,F,서울특별시 강동구 반포대거리
277,박영희,860924-1668295,1986-09-24,M,대구광역시 중구 강남대757로
283,최재호,710924-1211788,1971-09-24,M,전라북도 논산시 서초중앙6길
287,박정식,000923-4882878,2000-09-23,F,경상북도 고성군 압구정057가 (아름차면)


In [25]:
dup.이름.nunique(), dup.이름.unique(), dup.이름.value_counts()

(45,
 array(['박정식', '김서영', '최성진', '송영진', '김영식', '박승현', '김지원', '김지후', '김보람',
        '김우진', '김영환', '김숙자', '김진호', '김지은', '박우진', '박영희', '김정자', '김영자',
        '김정웅', '최순옥', '김영순', '김진우', '최재호', '김옥순', '이서연', '이아름', '이미정',
        '김승현', '홍정호', '최동현', '김예은', '김윤서', '김현주', '이민석', '이미경', '박진우',
        '박수민', '이수빈', '김성진', '김서윤', '김도현', '이정훈', '박서윤', '이선영', '박민수'],
       dtype=object),
 김지원    5
 박우진    4
 박정식    3
 김서영    3
 김예은    3
 김영순    3
 김윤서    3
 김영자    3
 김정자    3
 김승현    3
 김영식    3
 이수빈    2
 박수민    2
 박진우    2
 이미정    2
 김성진    2
 이민석    2
 김서윤    2
 김도현    2
 김현주    2
 이정훈    2
 박서윤    2
 이선영    2
 최동현    2
 홍정호    2
 이미경    2
 최재호    2
 이아름    2
 이서연    2
 최성진    2
 송영진    2
 박승현    2
 김지후    2
 김보람    2
 김우진    2
 김영환    2
 김숙자    2
 김진호    2
 김지은    2
 박영희    2
 김정웅    2
 최순옥    2
 김진우    2
 김옥순    2
 박민수    2
 Name: 이름, dtype: int64)

In [26]:
df1[df1.이름 == '김지원']

,이름,주민번호,생년월일,성별,주소
20,김지원,110924-4221730,2011-09-24,F,대구광역시 중구 역삼4가 (은서김동)
41,김지원,080923-3252841,2008-09-23,M,세종특별자치시 중랑구 반포대2거리
136,김지원,910924-1673374,1991-09-24,M,대구광역시 용산구 삼성6로 (승현강김면)
144,김지원,680923-1637306,1968-09-23,M,제주특별자치도 수원시 권선구 오금로
215,김지원,660924-1207833,1966-09-24,M,제주특별자치도 청양군 석촌호수가


In [27]:
#함수로 만들어볼까요
def check_dup_bycol(df, list):

    print(f'중복검사결과입니다.')

    return df[df.duplicated(subset=list, keep=False)].sort_values(by=list)

In [28]:
check_dup_bycol(df2,['요양기관기호'])

중복검사결과입니다.


,요양기관기호,병원이름,의사,면허번호,주소,의사 수,간호사 수,행정 직원 수,물리치료사 수,방사선사 수


## 3. groupby를 통한 데이터 집계

In [29]:
# 수진자 당 진료횟수, 금액, 평균진료금액, 기간은 2021년만
# groupby('컬럼명').agg(딕셔너리...{'컬럼명':'집계값','컬럼명':'집계값'})
df_exm.columns
df = df_exm[['주민번호', '이름','진료일자', '진료비']]
df.groupby('주민번호').agg({
    '이름':'first',
    '진료일자':['count','nunique'],
    '진료비':['sum','max','mean']
})
# groupby를 통해 반환된 결과는 또 다른 df이므로, 이를 이용하여 조건에 따른 필터링  가능함.

이름  진료일자              진료비                       
               first count nunique      sum     max           mean
주민번호                                                              
000923-4366119   김서영    12      12  2880000  420000  240000.000000
000923-4486589   이지민    10      10  2700000  470000  270000.000000
000923-4567446   문도윤     9       9  2070000  460000  230000.000000
000923-4882878   박정식     8       8  1120000  320000  140000.000000
000923-4897416   이수빈    14      14  2650000  420000  189285.714286
...              ...   ...     ...      ...     ...            ...
980924-2692288   한은정    15      15  3310000  450000  220666.666667
990924-1026747   김중수    13      13  2610000  470000  200769.230769
990924-1049700   김서영    10      10  2470000  480000  247000.000000
990924-1256214   박정식    12      12  2380000  490000  198333.333333
990924-2253910   서경자     9       9  3060000  480000  340000.000000

[240 rows x 6 columns]

### pandas의 group by
| 기능            | 함수명     |      | 기능            | 함수명     |
|-----------------|------------|------|-----------------|------------|
| 평균            | `mean`     |      | 표준편차        | `std`      |
| 합계            | `sum`      |      | 분산            | `var`      |
| 최소값          | `min`      |      | 중앙값          | `median`   |
| 최대값          | `max`      |      | 첫 번째 값      | `first`    |
| 갯수            | `count`    |      | 마지막 값       | `last`     |
| 고유한 값의 수  | `nunique`  |      | 최빈값          | `mode`     |
| 곱              | `prod`     |      | 평균 절대 편차  | `mad`      |
| 표준 오차      | `sem`      |      |         |        |
 

In [30]:
# 수진자 당 진료횟수, 금액, 평균진료금액, 기간은 2021년만
# groupby('컬럼명').agg(딕셔너리...{'컬럼명':'집계값','컬럼명':'집계값'})
df_exm.columns
df = df_exm[['주민번호', '이름','진료일자', '진료비', '수진자_시도']]
df_group = df.groupby(['주민번호','이름', '수진자_시도']).agg({
    # '이름':'first',
    # '병원이름':'first',
    '진료일자':['count','nunique'],
    '진료비':['sum','max','mean']
    
})
df_group
# groupby를 통해 반환된 결과는 또 다른 df이므로, 이를 이용하여 조건에 따른 필터링  가능함.

진료일자              진료비                       
                           count nunique      sum     max           mean
주민번호           이름  수진자_시도                                               
000923-4366119 김서영 서울특별시      12      12  2880000  420000  240000.000000
000923-4486589 이지민 서울특별시      10      10  2700000  470000  270000.000000
000923-4567446 문도윤 강원도         9       9  2070000  460000  230000.000000
000923-4882878 박정식 경상북도        8       8  1120000  320000  140000.000000
000923-4897416 이수빈 서울특별시      14      14  2650000  420000  189285.714286
...                          ...     ...      ...     ...            ...
980924-2692288 한은정 제주특별자치도    15      15  3310000  450000  220666.666667
990924-1026747 김중수 세종특별자치시    13      13  2610000  470000  200769.230769
990924-1049700 김서영 세종특별자치시    10      10  2470000  480000  247000.000000
990924-1256214 박정식 전라남도       12      12  2380000  490000  198333.333333
990924-2253910 서경자 전라남도        9       9  3060000  480000  340000.000000

[240 rows x 5 columns]

## 4. 수진자 지역별, 월별 진료비 현황
# 날짜기준 21.1분기~22.1분기
| 지역명   | 2021/Q1     |              | 2021/Q2     |              | 2021/Q3     |              |
|----------|-------------|--------------|-------------|--------------|-------------|--------------|
|          | 진료비      | 인원         | 진료비      | 인원         | 진료비      | 인원         |
|----------|-------------|--------------|-------------|--------------|-------------|--------------|
| 서울특별시 |             |              |             |              |             |              |
| 부산광역시 |             |              |             |              |             |              |
| 대구광역시 |             |              |             |              |             |              |
| 인천광역시 |             |              |             |              |             |              |
| 광주광역시 |             |              |             |              |             |              |


In [31]:
# 월별 구분을 위해 진료일자를 날짜 데이터 형식으로 변환
df_exm['진료일자'] = pd.to_datetime(df_exm['진료일자'])
df_exm.info()  # 0   진료일자     3000 non-null   datetime64[ns]    object에서 datetime으로 변환 확인


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   진료일자     3000 non-null   datetime64[ns]
 1   주민번호     3000 non-null   object        
 2   요양기관기호   3000 non-null   int64         
 3   병명코드     3000 non-null   object        
 4   진료비      3000 non-null   int64         
 5   이름       3000 non-null   object        
 6   생년월일     3000 non-null   object        
 7   성별       3000 non-null   object        
 8   주소_x     3000 non-null   object        
 9   병원이름     3000 non-null   object        
 10  의사       3000 non-null   object        
 11  면허번호     3000 non-null   int64         
 12  주소_y     3000 non-null   object        
 13  의사 수     3000 non-null   int64         
 14  간호사 수    3000 non-null   int64         
 15  행정 직원 수  3000 non-null   int64         
 16  물리치료사 수  3000 non-null   int64         
 17  방사선사 수   3000 non-null   int64   

| 기능          | 메서드/속성                           | 설명                                |
|---------------|---------------------------------------|-------------------------------------|
| 연도 추출     | `df['날짜'].dt.year`                  | 날짜에서 연도만 추출                |
| 월 추출       | `df['날짜'].dt.month`                 | 날짜에서 월만 추출                  |
| 일 추출       | `df['날짜'].dt.day`                   | 날짜에서 일만 추출                  |
| 요일 추출     | `df['날짜'].dt.dayofweek`             | 날짜에서 요일 추출 (0: 월요일)      |
| 요일 이름 추출| `df['날짜'].dt.day_name()`            | 요일의 이름 (예: 'Monday')          |
| 연도-월 추출  | `df['날짜'].dt.to_period('M')`        | 연도-월 추출 (YYYY-MM 형식)         |
| 연도-주 추출  | `df['날짜'].dt.to_period('W')`        | 연도-주 추출                        |
| 연도-분기 추출  | `df['날짜'].dt.to_period('Q')`        | 연도-분기 추출                        |
| 분기 추출     | `df['날짜'].dt.quarter`               | 날짜에서 분기 추출                  |
| 시간 추출     | `df['날짜'].dt.hour`                  | 날짜에서 시간 추출                  |
| 분 추출       | `df['날짜'].dt.minute`                | 날짜에서 분 추출                    |
| 날짜 차이 계산| `df['날짜'] - pd.to_datetime(기준일)` | 날짜 간의 차이 계산                 |
| 원하는 형식 변환 | `df['날짜'].dt.strftime('%Y/%m/%d')` | 날짜를 특정 형식으로 변환            |
| 월의 첫날     | `df['날짜'].dt.to_period('M').start_time` | 해당 월의 첫날 추출             |
| 월의 마지막날 | `df['날짜'].dt.to_period('M').end_time`   | 해당 월의 마지막날 추출          |


In [32]:
# 연도/분기 구분 컬럼을 넣어줍니다. 
df_exm['연도분기'] = df_exm['진료일자'].dt.to_period('Q')

In [33]:
df_exm.columns  ## 맨 끝에 '연도분기' 컬럼이 생성된 것을 확인.

Index(['진료일자', '주민번호', '요양기관기호', '병명코드', '진료비', '이름', '생년월일', '성별', '주소_x',
       '병원이름', '의사', '면허번호', '주소_y', '의사 수', '간호사 수', '행정 직원 수', '물리치료사 수',
       '방사선사 수', '병명', '수진자_시도', '수진자_구', '병원_시도', '병원_구', '연도분기'],
      dtype='object')

In [34]:
df_group = df_exm[['연도분기','주민번호','진료비','수진자_시도', '진료일자']]

# 필터링
# 날짜 (21.1분기~22.1분기)
con1 = '2021-01-01'
con2 = '2022-03-31'
#((df_group['진료일자'] >= con1) & (df_group['진료일자'] <= con2)).sum() # 1345행
df_group = df_group[(df_group['진료일자'] >= con1) &
                    (df_group['진료일자'] <= con2)]
df_group = df_group.groupby(['연도분기','수진자_시도']).agg({
    '주민번호':'count',
    '진료비':'sum'
})

df_group = df_group.reset_index()
df_group = df_group.pivot_table(columns='연도분기', index='수진자_시도',values=['주민번호','진료비'])

# MultiIndex 레벨 교환 (레벨 0과 레벨 1을 교환)
df_group.columns = df_group.columns.swaplevel(0, 1)

# 첫 번째 레벨(연도/분기)을 기준으로 컬럼 정렬
df_group = df_group.sort_index(axis=1, level=0)
# 두 번째 레벨(진료비, 주민번호)의 순서 조정 (진료비가 먼저 오도록)
df_group = df_group.reindex(columns=['진료비', '주민번호'], level=1)
# 컬럼명 변경
df_group.rename(columns={'주민번호': '진료인원'}, inplace=True)
df_group

연도분기        2021Q1           2021Q2           2021Q3           2021Q4        \
               진료비  진료인원        진료비  진료인원        진료비  진료인원        진료비  진료인원   
수진자_시도                                                                        
강원도      1450000.0   5.0   160000.0   1.0  2430000.0   9.0  1870000.0   6.0   
경기도      1220000.0   5.0  2800000.0  10.0  1030000.0   4.0  2930000.0   9.0   
경상남도      860000.0   2.0  1020000.0   3.0   860000.0   3.0  1460000.0   5.0   
경상북도     1760000.0   6.0   890000.0   4.0  1730000.0   5.0   600000.0   4.0   
광주광역시     910000.0   3.0  2290000.0   8.0  2080000.0   6.0  1610000.0   8.0   
대구광역시    2820000.0  13.0  5170000.0  20.0  3240000.0  14.0  2870000.0  12.0   
대전광역시    4750000.0  17.0  1420000.0   7.0  3600000.0  12.0   810000.0   5.0   
부산광역시     720000.0   3.0   680000.0   3.0  1650000.0   8.0  1750000.0   7.0   
서울특별시    2280000.0   8.0  1820000.0  10.0  1480000.0   9.0  3010000.0  12.0   
세종특별자치시  4790000.0  15.0  2350000.0  12.0  2220000.0   8.0  1970000.0   7.0   
울산광역시    1940000.0   7.0  2520000.0  12.0   880000.0   5.0  1100000.0   5.0   
인천광역시    2220000.0   9.0  2620000.0   7.0  1860000.0   9.0  2140000.0   9.0   
전라남도     1130000.0   6.0  1230000.0   5.0  3030000.0   9.0  5870000.0  22.0   
전라북도     3060000.0  13.0  1850000.0   7.0  2370000.0  11.0  3430000.0  10.0   
제주특별자치도  5570000.0  18.0  2120000.0  10.0  5390000.0  18.0  2520000.0  13.0   
충청남도       90000.0   3.0  1810000.0   5.0  1870000.0   7.0  1860000.0   7.0   
충청북도     1660000.0   6.0   830000.0   3.0   900000.0   6.0  1030000.0   4.0   

연도분기        2022Q1        
               진료비  진료인원  
수진자_시도                    
강원도       900000.0   5.0  
경기도      3460000.0  13.0  
경상남도     1010000.0   3.0  
경상북도     1950000.0   6.0  
광주광역시    1280000.0   5.0  
대구광역시    1020000.0   4.0  
대전광역시    4170000.0  17.0  
부산광역시          NaN   NaN  
서울특별시    1580000.0   8.0  
세종특별자치시  2420000.0  10.0  
울산광역시    2510000.0   9.0  
인천광역시    3430000.0  12.0  
전라남도     1670000.0   7.0  
전라북도     3610000.0  15.0  
제주특별자치도  3420000.0  11.0  
충청남도     1940000.0   7.0  
충청북도     2630000.0  14.0

## 4. 상위 k, 상위 k% 추출

2023년간 직원 수 상위 병원 k개, 수입 상위 k% 병원 추출

In [35]:
#  날짜 데이터로 변경
df_exm['진료일자'] = pd.to_datetime(df_exm['진료일자'])
df_exm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   진료일자     3000 non-null   datetime64[ns]
 1   주민번호     3000 non-null   object        
 2   요양기관기호   3000 non-null   int64         
 3   병명코드     3000 non-null   object        
 4   진료비      3000 non-null   int64         
 5   이름       3000 non-null   object        
 6   생년월일     3000 non-null   object        
 7   성별       3000 non-null   object        
 8   주소_x     3000 non-null   object        
 9   병원이름     3000 non-null   object        
 10  의사       3000 non-null   object        
 11  면허번호     3000 non-null   int64         
 12  주소_y     3000 non-null   object        
 13  의사 수     3000 non-null   int64         
 14  간호사 수    3000 non-null   int64         
 15  행정 직원 수  3000 non-null   int64         
 16  물리치료사 수  3000 non-null   int64         
 17  방사선사 수   3000 non-null   int64   

In [36]:
df_exm['직원수'] = df_exm[['의사 수', '간호사 수', '행정 직원 수', '물리치료사 수','방사선사 수']].sum(axis=1)
df_exm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 2999
Data columns (total 25 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   진료일자     3000 non-null   datetime64[ns]
 1   주민번호     3000 non-null   object        
 2   요양기관기호   3000 non-null   int64         
 3   병명코드     3000 non-null   object        
 4   진료비      3000 non-null   int64         
 5   이름       3000 non-null   object        
 6   생년월일     3000 non-null   object        
 7   성별       3000 non-null   object        
 8   주소_x     3000 non-null   object        
 9   병원이름     3000 non-null   object        
 10  의사       3000 non-null   object        
 11  면허번호     3000 non-null   int64         
 12  주소_y     3000 non-null   object        
 13  의사 수     3000 non-null   int64         
 14  간호사 수    3000 non-null   int64         
 15  행정 직원 수  3000 non-null   int64         
 16  물리치료사 수  3000 non-null   int64         
 17  방사선사 수   3000 non-null   int64   

In [37]:
## 자료발췌
#기간
y = 2023
df_exm[df_exm['진료일자'].dt.year == y]  # 591행

# 병원기준
df_group = df_exm[['연도분기', '요양기관기호','병원이름','주민번호','병명코드','병원_시도','진료비','직원수']]
df_group = df_group.groupby(['요양기관기호','병원이름']).agg({
    '직원수':'sum',
    '진료비':'sum',
    '병원_시도':'first'    
})
df_group

,,직원수,진료비,병원_시도
요양기관기호,병원이름,,,
10039659,주식회사 류이우병원,1012,5060000,광주광역시
10502084,(주) 권병원,1140,8290000,울산광역시
10815376,김민김병원,1012,6350000,전라북도
10823075,유한회사 엄최이병원,930,8860000,경상북도
11051510,김문김병원,840,6910000,울산광역시
...,...,...,...,...
38687258,박문병원,660,7930000,대전광역시
39324705,박황곽병원,945,6680000,대구광역시
39349156,조이병원,800,7120000,광주광역시


In [38]:
df_group.sort_values(by='직원수', ascending=False)
df_group.head(10)

,,직원수,진료비,병원_시도
요양기관기호,병원이름,,,
10039659,주식회사 류이우병원,1012,5060000,광주광역시
10502084,(주) 권병원,1140,8290000,울산광역시
10815376,김민김병원,1012,6350000,전라북도
10823075,유한회사 엄최이병원,930,8860000,경상북도
11051510,김문김병원,840,6910000,울산광역시
11294420,이김병원,1824,9120000,경상남도
11853692,박이병원,1064,9880000,충청남도
11892955,(주) 최병원,1476,10410000,부산광역시
11989261,주식회사 이심임병원,968,5160000,경상북도


In [39]:
# 상위 5% 병원 수 계산
k = int(len(df) * 0.05)
#이하 동일